## Set up environment

In [1]:
from dotenv import load_dotenv

_ = load_dotenv(override=True)

In [2]:
import os
from common.context import LLMTagPredictionContext


EXPERIMENT_ID = "2024-10-21-try-llm-to-predict-tighthole"
RUN_ID = "13-give-definition-and-ask-justification"


CONTEXT = LLMTagPredictionContext(
    description="Try asking LLM to assess all tags at once. Give examples.",
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
    tags_in_scope=sorted(
        [
            "tighthole",
        ]
    ),
    llm_model=os.environ["AZURE_OPENAI_DEPLOYMENT_ID"],
    with_notags=True,
)

## Fetch datasets

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from common.datasets import load_input_dataset

dataset_name = "reviewed_distributed_ddr_v3.csv"

dataset_df = load_input_dataset(
    dataset_name,
    columns_to_convert_to_sets=["tags", "Reviewed tags"],
)
CONTEXT.used_datasets = [dataset_name]

# dataset_df = dataset_df.sample(frac=1, random_state=43).reset_index(drop=True)  # shuffle so that a prefix of the dataset is more representative of the whole dataset
# train_df, test_df = train_test_split(dataset_df, test_size=0.75, random_state=42)
# dataset_df = train_df

# dataset_df = pd.concat([
#     dataset_df[dataset_df['Reviewed tags'].apply(lambda tags: 'tighthole' in tags)],
#     # dataset_df[dataset_df['Reviewed tags'].apply(lambda tags: 'tighthole' not in tags)][:500],
#     dataset_df[dataset_df['id'].apply(lambda id: id in problematic_ids)],
# ]).drop_duplicates('id')
dataset_df

,id,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN
...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\nMeanwhile:...,INTERV,N,SAFETY,{},YES,{},NaN
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,{lostcirculation},NaN


## Apply the model

In [4]:
import pandas as pd
from common.llm import ask_openai

examples = dataset_df.sort_values(by="Text", key=lambda x: x.str.len())
examples = examples[examples["Text"].str.len() > 40]
examples = examples[examples["Text"].str.len() < 200]

example_list = [
    examples[examples["Reviewed tags"].apply(lambda tags: tag in tags)][
        ["Text", "Reviewed tags", "Comments"]
    ]
    for tag in CONTEXT.tags_in_scope
]
example_list.append(
    examples[examples["Reviewed tags"].apply(lambda tags: 'tighthole' not in tags)][examples["tags"].apply(lambda tags: 'tighthole' in tags)][
        ["Text", "Reviewed tags", "Comments"]
].head(len(example_list[0])))
example_list.append(
    examples[examples["Reviewed tags"].apply(lambda tags: 'tighthole' not in tags)][
        ["Text", "Reviewed tags", "Comments"]
].head(40))

# Concatenate the examples into a single DataFrame
examples = pd.concat(example_list, ignore_index=True)
# Convert 'Reviewed tags' to a sorted list
examples["Reviewed tags"] = examples["Reviewed tags"].apply(
    lambda tags: tuple(sorted(tags))
)
# Drop duplicate rows
examples = examples.drop_duplicates()
examples = examples.sort_values(by="Text")
examples

/var/folders/45/5727m82n46s5tn66h72y7gzh0000gn/T/ipykernel_43224/884685348.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  examples[examples["Reviewed tags"].apply(lambda tags: 'tighthole' not in tags)][examples["tags"].apply(lambda tags: 'tighthole' in tags)][


,Text,Reviewed tags,Comments
8,Attempted to POOH from 370m. Tight hole from 3...,"(tighthole,)",NaN
26,"Attempted to RIH with casing, no go. Set down ...","(stuckpipe, tighthole)",NaN
16,Attempted to free fish with max 180 Mt over pu...,"(stuckpipe, tighthole)",NaN
35,Attempted to pass restriction at 1170 m severa...,"(holecleaning, tighthole)","Should aslo be"" tighthole"" ""Attempted to pass ..."
27,"Attempted to pull 5.5"" m tbg fish#5 (98 m) fre...","(tighthole,)",NaN
...,...,...,...
74,Troubleshot stuck finger on on fingerboard.,"(surfeqfailure,)",NaN
76,Waited for the BJ rep to get his 8 hour rest.,"(wait,)",NaN
7,Worked pipe out of hole. Unable to pass restri...,"(tighthole,)",NaN
29,Worked pipe with with increasing parameters to...,"(stuckpipe, tighthole)",NaN


In [5]:
new_message = '\n'.join(
    f"# Report\n\n{ex['Text']}\n\ntags: {', '.join(sorted(ex['Reviewed tags'] or ['no tags']))}\n" + (f'comments: {ex["Comments"]}\n' if not pd.isna(ex["Comments"]) else '')
    for ex in examples.to_dict(orient="records")
)
print(new_message)


# Report

Attempted to POOH from 370m. Tight hole from 365m. Took 35 KLBS O/P. Worked string twice, no improvements.

tags: tighthole

# Report

Attempted to RIH with casing, no go. Set down 60 - 65 ton at 1995 m. Worked string and pushed down to 2005 m wwith 45 - 60 ton weight. Unable to pass 2005 m.

tags: stuckpipe, tighthole

# Report

Attempted to free fish with max 180 Mt over pull while jarring. Jarred a total of 17 times with max 60 Mt. No movement observed.

tags: stuckpipe, tighthole

# Report

Attempted to pass restriction at 1170 m several times with up to 20 MT overpull - Observed indication of dragging BHA into foreign object or cuttings bed. RIH to 1198 m.

tags: holecleaning, tighthole
comments: Should aslo be" tighthole" "Attempted to pass restriction at 1170 m several times with up to 20 MT overpull"

# Report

Attempted to pull 5.5" m tbg fish#5 (98 m) free with straight pull.  Worked string between 225 klbs - 560 klbs HL (Max 350 klbs overpull). - Fish moved: 2 cm.


In [6]:

response = ask_openai(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    deployment_name='gpt-4o',
    system_prompt="""
You will be given many drilling reports and their tags. Some of these reports will be describe a Tight Hole scenario. Such reports will have a 'tighthole' tag.
Based on these reports, define what is and what isn't the Tight Hole.""",
    prompt=new_message,
)

print(response)

Based on the reports provided, a "Tight Hole" scenario generally involves:

1. **Overpull**: The need to apply significant overpull to move the drill string or casing, indicating resistance or obstruction in the wellbore.
2. **Resistance or Obstruction**: Difficulty in moving equipment through the wellbore due to tight spots, restrictions, or obstructions.
3. **Working the String**: Efforts to work the string back and forth to overcome resistance.
4. **Jarring**: Use of jarring techniques to attempt to free stuck equipment.
5. **Reaming**: Reaming through tight spots to clear the path.
6. **Lubricating or Washing**: Using fluid circulation to assist in moving through tight spots.

What isn't considered a "Tight Hole" scenario:

1. **Equipment Failure**: Issues related to surface equipment or downhole tool failures without mention of wellbore resistance.
2. **Routine Operations**: Standard operations like running in hole or pulling out without mention of resistance.
3. **Wait Times**: D

In [7]:
# """
# You will be given a daily drilling report written by an engineer.
# Please determine whether the report describes a Tight Hole scenario.

# Tight Hole refers to a situation of a reduced wellbore diameter. It causes the drill string encounter increased friction or resistance as it's being run into or pulled out of the hole. This can happen due to various reasons, such as cuttings settling around the drill pipe, formation swelling, differential sticking, or borehole collapse.

# To tackle a Tight Hole, an engineer would typically take several steps:
# - Work the string: work the drill string up and down (reciprocating) or rotating it to try and loosen the resistance. This can help clear obstructions like cuttings or debris around the drill pipe.
# - Circulation: The engineer may also circulate drilling fluid (mud) through the hole to help remove cuttings or debris that may be causing the tight spot.
# - Ream the hole: If reciprocating and circulation don't work, reaming the hole (enlarging it by running a reamer or stabilizer tool) can remove tight spots or cuttings build-up.

# The drilling report would often describe inability to pass the drilling equipment (e.g. string or whipstock) through a tight spot. It can mention overpull (OP) of the string and increased drag as the engineer tries to tackle the Tight Hole situation.

# Other scenarios that might occur during drilling: Lost Circulation, Hard Drilling, Wellbore Stability, Stuck Pipe, Well Control, Hole Cleaning, Boulders, Shallow Gas, Shallow Water, Wellbore Breathing, Pack-Off, Directional Control, Low ROP, High ROP, Downhole Equipment Failure, Wait, Surface Equipment Failure.
# These scenarios may or may not occur at the same time with Tight Hole. Tight Hole should not be confused with them.

# Do not confuse Tight Hole with Stuck Pipe, when the drill string or casing becomes immobilized in the wellbore and can not be freed.

# The drill string may also be called a cable or a wire.

# Respond in 2 lines:
# 1. Think step by step: does the report describe a Tight Hole scenario?
# 2. Write "tighthole" if the report describes a Tight Hole scenario, and "notag" otherwise."""

In [8]:
SYSTEM_PROMPT = f"""
You will be given a daily drilling report.
Please determine whether the situation "Tight Hole" is described in it.

Definition of "Tight Hole": Reduced wellbore diameter. The reason for this can be wellbore stability or geometrical challenges in the trajectory. It could also be due to mechanical issues where large equipment is ran into a small hole.

The drilling report where "Tight Hole" occured often mentions overpull (or OP for short), increased drag, or inability to pass through a tight spot.

Other situations that might occur during drilling: Lost Circulation, Hard Drilling, Wellbore Stability, Stuck Pipe, Well Control, Hole Cleaning, Boulders, Shallow Gas, Shallow Water, Wellbore Breathing, Pack-Off, Directional Control, Low ROP, High ROP, Downhole Equipment Failure, Wait, Surface Equipment Failure.
These situations may or may not occur at the same time with Tight Hole. Tight Hole should not be confused with them.

Respond in 2 lines:
1. Explain the drilling report. Reason whether it describes a Tight Hole situation.
2. Write "tighthole" if the tag applies and "notag" if the tag doesn't apply.
""".strip()

# for idx, (_, row) in enumerate(examples.iterrows()):
#     SYSTEM_PROMPT += f"\n## Example drilling report {idx}\nText: {row['Text']}\n\n## Correct response\n{', '.join(row['Reviewed tags'])}\n"

CONTEXT.llm_system_prompt = SYSTEM_PROMPT

print(SYSTEM_PROMPT)
print(len(SYSTEM_PROMPT))

You will be given a daily drilling report.
Please determine whether the situation "Tight Hole" is described in it.

Definition of "Tight Hole": Reduced wellbore diameter. The reason for this can be wellbore stability or geometrical challenges in the trajectory. It could also be due to mechanical issues where large equipment is ran into a small hole.

The drilling report where "Tight Hole" occured often mentions overpull (or OP for short), increased drag, or inability to pass through a tight spot.

Other situations that might occur during drilling: Lost Circulation, Hard Drilling, Wellbore Stability, Stuck Pipe, Well Control, Hole Cleaning, Boulders, Shallow Gas, Shallow Water, Wellbore Breathing, Pack-Off, Directional Control, Low ROP, High ROP, Downhole Equipment Failure, Wait, Surface Equipment Failure.
These situations may or may not occur at the same time with Tight Hole. Tight Hole should not be confused with them.

Respond in 2 lines:
1. Explain the drilling report. Reason whethe

In [9]:
TEMPERATURE = 0

CONTEXT.llm_temperature = TEMPERATURE

In [10]:
from concurrent.futures import ThreadPoolExecutor
import os
from tqdm.auto import tqdm

from common.llm import ask_openai

def build_message(df_row: dict):
    phase = df_row['phase']
    code = df_row['code']
    subcode = df_row['subCode']
    text = df_row['Text']
    return F"Phase: {phase}\nCode: {code}\nSubcode: {subcode}\n\n{text}"


# Define a function to call ask_openai and get the predicted tags
def get_predicted_tags(row):
    message = build_message(row)
    try:
        response = ask_openai(
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_OPENAI_KEY"],
            api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_ID"],
            system_prompt=SYSTEM_PROMPT,
            prompt=message,
        )
    except Exception as e:
        if "content management policy. Please modify your prompt" in str(e):
            print(e)
            return []  # running into the content filter
        raise

    in_scope = set(CONTEXT.tags_in_scope)

    def normalize_tag(t):
        # sometimes model makes mistakes
        t = t.lower()
        t = t.strip('12.')
        if t.startswith("tags:"):
            t = t[len("tags:") :]
        t = t.strip().strip("()")
        if t not in in_scope and t != 'notag':
            print('Unexprected tag:', t)
        return t
    
    # justification, tag = '', response
    import string
    response = response.strip().strip(string.punctuation)
    if '\n' in response:
        justification, tag = response.strip().rsplit('\n', 1)
    else:
        justification, tag = response.strip().rsplit(None, 1)

    tag = normalize_tag(tag)
    return tag, justification

assessed_df = dataset_df.copy()


def parallel_apply(df, func, num_threads: int):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(tqdm(executor.map(func, df.to_dict(orient="records")), total=len(df)))
    return results


assessed_df[["Predicted", "Justification"]] = parallel_apply(
    assessed_df, get_predicted_tags, num_threads=3
)
assessed_df

/Users/gukov/.pyenv/versions/3.12.2/envs/global3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1442/1442 [05:13<00:00,  4.60it/s]


,id,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,Predicted,Justification
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,notag,1. The drilling report describes a situation w...
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,notag,The drilling report describes a situation wher...
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,notag,1. The drilling report describes a situation w...
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,notag,The drilling report describes a situation wher...
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,notag,1. The drilling report describes activities re...
...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\nMeanwhile:...,INTERV,N,SAFETY,{},YES,{},NaN,tighthole,1. The drilling report indicates attempts to p...
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN,notag,1. The drilling report indicates normal drilli...
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN,notag,The drilling report indicates circulating out ...
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,{lostcirculation},NaN,notag,The drilling report indicates a loss of OBM an...


In [11]:
# nothing to do, DDR tagging using regex rules is already applied to the dataset in this experiment
from common.assessment import expand_tags

assessed_df = expand_tags(
    assessed_df,
    tags_in_scope=CONTEXT.tags_in_scope,
    ground_truth_tags_column="Reviewed tags",
    predicted_tags_column="Predicted",
)
assessed_df

,id,Text,phase,code,subCode,tags,Are tags correct?,Comments,Justification,expected__tighthole,actual__tighthole
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,NaN,1. The drilling report describes a situation w...,False,False
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,NaN,The drilling report describes a situation wher...,False,False
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,NaN,1. The drilling report describes a situation w...,False,False
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,NaN,The drilling report describes a situation wher...,False,False
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,NaN,1. The drilling report describes activities re...,False,False
...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\nMeanwhile:...,INTERV,N,SAFETY,{},YES,NaN,1. The drilling report indicates attempts to p...,False,True
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,NaN,1. The drilling report indicates normal drilli...,False,False
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,NaN,The drilling report indicates circulating out ...,False,False
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,NaN,The drilling report indicates a loss of OBM an...,False,False


In [12]:
# problematic_ids = {
#     item['id']
#     for item in assessed_df.to_dict(orient="records")
#     if (item['expected__tighthole'] and not item['actual__tighthole']) or (item['actual__tighthole'] and not item['expected__tighthole'])
# }
# len(problematic_ids)

## Evaluate predicted tags

In [13]:
from common.evaluation import TagMatchingEvaluator

evaluator = TagMatchingEvaluator(
    assessed_df=assessed_df,
    tags_in_scope=CONTEXT.tags_in_scope,
    with_notags=CONTEXT.with_notags,
)

In [14]:
evaluator.eval_per_tag()

,tag,precision,recall,f1,true_positives,positives_in_ground_truth,negatives_in_ground_truth
0,tighthole,0.452261,0.9,0.602007,90,100,1342


In [15]:
evaluator.eval_individual_ddrs()

,id,Text,phase,code,subCode,tags,Are tags correct?,Comments,Justification,expected__tighthole,actual__tighthole,expected__notags,actual__notags,precision,recall,f1,true_positives
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,NaN,1. The drilling report describes a situation w...,False,False,True,True,1.0,1.0,1.0,1
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,NaN,The drilling report describes a situation wher...,False,False,True,True,1.0,1.0,1.0,1
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,NaN,1. The drilling report describes a situation w...,False,False,True,True,1.0,1.0,1.0,1
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,NaN,The drilling report describes a situation wher...,False,False,True,True,1.0,1.0,1.0,1
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,NaN,1. The drilling report describes activities re...,False,False,True,True,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\nMeanwhile:...,INTERV,N,SAFETY,{},YES,NaN,1. The drilling report indicates attempts to p...,False,True,True,False,0.0,0.0,0.0,0
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,NaN,1. The drilling report indicates normal drilli...,False,False,True,True,1.0,1.0,1.0,1
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,NaN,The drilling report indicates circulating out ...,False,False,True,True,1.0,1.0,1.0,1
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,NaN,The drilling report indicates a loss of OBM an...,False,False,True,True,1.0,1.0,1.0,1


In [16]:
evaluator.average_metrics()

,Type,precision,recall,f1
0,Average per DDR,0.917476,0.917476,0.917476
1,Average per Tag,0.452261,0.900000,0.602007


## Save evaluation report

In [17]:
from common.datasets import save_evaluation_report

save_evaluation_report(
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
    dataset_df=dataset_df,
    assessed_df=assessed_df,
    evaluator=evaluator,
    context=CONTEXT,
)